In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [0]:
# Utils
import os
import torch
import torch.optim 

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix
    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))
    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def get_optimizer(model, state_dict=None):
    optimizer = None
    if optim == 'sgd':
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=lr,
            momentum=0.9,
            weight_decay=1e-5,
            nesterov=True
        )
    elif optim == 'adam':
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=lr
        )
    elif optim == 'rmsprop':
        pass
    if state_dict:
        optimizer.load_state_dict(state_dict)
    return optimizer

# from imagenet example of pytorch: https://github.com/pytorch/examples/blob/master/imagenet/main.py
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def adjust_learning_rate(optimizer, epoch,  steps=(20, 40), dec_rate=0.1):
    """
    Decreases the learning rate to the initial LR decayed by dec_rate every given step
    If steps is an integer, decreases lr every <steps> epoch by dec_rate
    """
    assert type(steps) in [list, tuple, int]
    changed_flag = False
    lr = optimizer.param_groups[0]['lr']
    if type(steps) == int:
        if epoch % steps == 0:
            changed_flag = True
    else:
        if epoch in steps:
            changed_flag = True
    if changed_flag:
        lr *= dec_rate
        for param_group in optimizer.param_groups:
            print('Decreasing learning rate to {:1.5f}'.format(lr))
            param_group['lr'] = lr
    # step = 20
    # if lr * (0.1 ** (epoch // step)) > 1e-5:
    #     lr = lr * (0.1 ** (epoch // step))
    #     for param_group in optimizer.param_groups:
    #         if param_group['lr'] != lr:
    #             print('Decreasing learning rate to {:1.5f}'.format(lr))
    #         param_group['lr'] = lr

def save_checkpoint(states, output_dir, is_best=False, filename='checkpoint.pth'):
    if is_best:
        torch.save(states, filename + '_model_best.pth')
    else:
        torch.save(states, filename + '.pth')


In [0]:
# Initialize paths
import os.path as osp
import sys

def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)

this_dir = osp.dirname('__file__')
lib_path = osp.join(this_dir, '..')
add_path(lib_path)


In [0]:
# functions
import os
import json
import time
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
# from utils import _init_paths
from sklearn.metrics import confusion_matrix

def train_epoch(epoch, train_loader, model, criterion, optimizer, use_cuda=True):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader), batch_time, data_time,
        top1, top5, losses, prefix="Epoch: [{}]".format(epoch + 1))
    print_freq = len(train_loader) // 4 + 1
    all_preds = []
    all_labels = []
    model.train()
    end = time.time()
    for i, (paths, inputs, labels) in enumerate(train_loader):
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        data_time.update(time.time() - end)
        # forward + backward + optimize
        if type(model).__name__ == 'Inception3' and model.aux_logits:
            outputs, aux_outputs = model(inputs)
            loss_aux = criterion(aux_outputs, labels)
            loss_final = criterion(outputs, labels)
            loss = loss_final + 0.4*loss_aux
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(acc1[0], inputs.size(0))
        top5.update(acc5[0], inputs.size(0))
        # for confusion matrix calculation
        _, preds = outputs.topk(1, 1, True, True)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        # print statistics
        if i % print_freq == 0 or i + 1 == len(train_loader):
            progress.print(i+1)
    print(confusion_matrix(all_labels, all_preds))
    return top1.avg, top5.avg

def validate_epoch(val_loader, model, criterion, use_cuda=True):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = utils.ProgressMeter(len(val_loader), batch_time, top1, top5, losses,
                                   prefix='Val: ')
    # switch to evaluate mode
    all_preds = []
    all_labels = []
    model.eval()
    print_freq = len(val_loader) // 4 + 1
    with torch.no_grad():
        end = time.time()
        for i, (_, inputs, labels) in enumerate(val_loader):
            if use_cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # measure accuracy and record loss
            acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1[0], inputs.size(0))
            top5.update(acc5[0], inputs.size(0))
            # for confusion matrix calculation
            _, preds = outputs.topk(1, 1, True, True)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            if i % print_freq == 0 or i+1 == len(val_loader):
                progress.print(i+1)
        print(confusion_matrix(all_labels, all_preds))
        return top1.avg, top5.avg

def test_cassava(test_loader, model, class_names):
    end = time.time()
    model.eval()
    preds = []
    image_names = []
    softmax_outs = []
    report_freq = len(test_loader) // 9 + 1
    with torch.no_grad():
        # test is done with batch_size=1
        for i, (path, inputs, label) in enumerate(test_loader):
            # print(path)
            image_names.append(path[0].split('/')[-1])
            if use_cuda:
                inputs = inputs.cuda()
            if tencrop_test:
                bs, ncrops, c, h, w = inputs.size()
                outputs = model(inputs.view(-1, c, h, w))  # fuse batch size and ncrop
                outputs = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
            else:
                outputs = model(inputs)
            softmax_outs.extend(F.softmax(outputs).cpu().numpy())
            _, pred = outputs.topk(1, 1, True, True)
            preds.append(class_names[pred])

            if i % report_freq == 0 or i+1 == len(test_loader):
                print('Processed test data: ', i+1)
    results_dict = {'Category': preds, 'Id': image_names}
    results_df = pd.DataFrame(results_dict)
    submission_file = 'Epoch{}_{}_{}_{}_{}{}{}'.format(
        num_epochs, model_input_size, arch,
        optim, batch_size, '_subset' if subset_finetune else '',
        '_weightedloss' if use_weighted_loss else '')
    i = 1
    # if there is another file with same name, find a new name using i variable in filename
    while os.path.exists(str(i) + '_' + submission_file + '.csv'):
        i += 1
    # export prediction in requested kaggle competition format
    results_df.to_csv(str(i) + '_' + submission_file + '.csv', index=False)
    # save softmax outputs for later ensembling
    np.save(str(i) + submission_file, np.asarray(softmax_outs))

In [0]:
import os
import json
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from torchvision.datasets import ImageFolder

def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.
    Args:
        filename (string): path to a file
        extensions (iterable of strings): extensions to consider (lowercase)
    Returns:
        bool: True if the filename ends with one of given extensions
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)

# Adapted from pytorch folder.py that contains DatasetFolder and ImageFolder
def make_dataset(root, paths_dict, class_to_idx, extensions):
    """
    Generates (path, label) tuples for each of the sample and returns as a list
    :param root: root folder of given dataset split
    :param paths_dict: contains class names as keys and image paths as values
    :param class_to_idx: class to index dictionary
    :param extensions: extensions to distinguish images
    :return: images as tuple in form (path, class id)
    """
    images = []
    root = os.path.join(root, 'train')
    root = os.path.expanduser(root)
    for target in sorted(class_to_idx.keys()):
        class_dir = os.path.join(root, target)
        if not os.path.isdir(class_dir):
            continue
        for fname in sorted(paths_dict[target]):
            if has_file_allowed_extension(fname, extensions):
                path = os.path.join(class_dir, fname)
                item = (path, class_to_idx[target])
                images.append(item)
    return images

def _create_paths_dict(root_dir, split, split_percentage, seed=None):
    """
    :param root_dir:
    :param split: split type 'train' or 'val
    :param split_percentage:
    :return paths_dict: a dictionary consisting of class names as
                        keys and image paths as a list for each key
    """
    split_dir = os.path.join(root_dir, 'train')  # get both train and validation data from train dir
    class_names = sorted(os.listdir(split_dir))
    class_names = [ch for ch in class_names if '.' not in ch]  # remove folder name if contains dot(.) in
    paths_dict = dict()
    total = 0
    for class_name in class_names:
        # for consistent random split generation between runs, set seed
        np.random.seed(seed)
        class_dir = os.path.join(split_dir, class_name)
        image_names = sorted(os.listdir(class_dir))
        num_images = len(image_names)
        total += num_images
        all_indices = np.array(np.random.permutation(range(num_images)))
        if split == 'val':  # given percentage is for val, so split for train and get the rest
            split_percentage = 1 - split_percentage  # get train set split percentage
        split_border = np.int(num_images * split_percentage)
        split_ind = all_indices[:split_border] if split == 'train' else all_indices[split_border:]
        split_paths = [os.path.join(class_dir, image_names[i]) for i in split_ind]
        paths_dict[class_name] = split_paths
    return paths_dict

def _compute_class_weights(paths_dict, class_to_idx):
    y_train = []
    for class_name in sorted(paths_dict.keys()):
        y_train.extend([class_to_idx[class_name]] * len(paths_dict[class_name]))
    class_weights = compute_class_weight('balanced', np.unique(y_train),  y_train)
    return class_weights

class CassavaTestFolder(ImageFolder):
    def __init__(self, root, transform=None):
        super(CassavaTestFolder, self).__init__(root=root, transform=transform)
    def __getitem__(self, item):
        path, target = self.imgs[item]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        return path, img, target

class CassavaFolder(ImageFolder):
    def __init__(self, root, split, split_percentage=0.8, transform=None):
        """
        A variation of ImageFolder class of pytorch adapted to cassava dataset
        :param root: root directory of cassava dataset that contains train-test-extraimages splits
        :param split: 'train', 'val' or 'extraimages'
        :param split_percentage: 'used only for 'train' and 'val' sets if needed
        :param transform: a series of pytorch transforms to apply to images
        """
        super(CassavaFolder, self).__init__(root=root, transform=transform)
        if split in ['train', 'val']:
            # given a seed for generating the same train and val splits in different runs
            paths_dict = _create_paths_dict(root, split, split_percentage, seed=12)
        else:  # extraimages
            # extraimages are classified by a model that scores 91.456 on public leaderboard
            # only samples which are classified above a given confidence threshold are considered
            extraimages_threshold = 0.99
            extraimages_json = 'extraimages_above_threshold_{}.json'.format(extraimages_threshold)
            extraimages_json = os.path.join(root, extraimages_json)
            with open(extraimages_json, 'r') as fp:
                paths_dict = json.load(fp)
        self.classes = sorted(paths_dict.keys())
        self.class_to_idx = dict(zip(self.classes, range(len(self.classes))))
        self.samples = self.imgs = make_dataset(root, paths_dict, self.class_to_idx, self.extensions)
        self.class_weights = _compute_class_weights(paths_dict, self.class_to_idx)
    def __getitem__(self, item):
        """
        Returns path of the image as extra to ImageFolder class
        :param item:
        :return:
        """
        path, target = self.imgs[item]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        return path, img, target


In [0]:
# Load the data
import os
import torch
import torchvision.transforms as transforms
# from dataloaders import cassava_folder

# pre-calculated from training data
mean_v = [0.4478, 0.4967, 0.3218]
std_v = [0.2053, 0.2062, 0.1792]

competition_root_path = '/content/drive/My Drive/Kaggle/data'


def get_dataloader(data_split, train_percentage=0.8):
    # initialize datasets and dataloaders
    # resize_res: 256 for 224, 512 for 448, 640 for 560 (to crop from a higher resolution)
    resize_res = int(model_input_size * 1000 / 875)
    print('Transform resize resolution: ', resize_res)
    mean_vec = mean_v
    std_vec = std_v
    dataset = loader = None
    dir_path = os.path.dirname('__file__')
    if data_split == 'train':
        train_transform = transforms.Compose([
            transforms.RandomRotation(15),
            transforms.RandomResizedCrop(model_input_size),
            transforms.RandomHorizontalFlip(),
            # transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_vec, std=std_vec)
        ])
        dataset = CassavaFolder(
            root=dir_path + competition_root_path, split='train',
            split_percentage=train_percentage, transform=train_transform)
        num_train_samples = len(dataset)
        if use_extraimages:
            extra_dataset = CassavaFolder(
                root=dir_path + competition_root_path, split='extraimages',
                transform=train_transform)
            print("Number of extra samples: ", len(extra_dataset))
            dataset = torch.utils.data.ConcatDataset[dataset, extra_dataset]
            dataset.classes = dataset[0].classes
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size,
            shuffle=True, num_workers=4, pin_memory=True)
        print("Number of training samples: ", num_train_samples)
        if use_extraimages:
            print("Number of combined training samples: ", len(dataset))
        print("Number of classes: ", len(dataset.classes))
    elif data_split == 'val':
        val_transform = transforms.Compose([
            transforms.Resize(resize_res),
            transforms.CenterCrop(model_input_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_vec, std=std_vec)
        ])
        dataset = CassavaFolder(
            root=dir_path + competition_root_path, split='val',
            split_percentage=1-train_percentage, transform=val_transform)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size,
            shuffle=False, num_workers=4, pin_memory=True)
        print("Number of validation samples: ", len(dataset))
        print("Number of classes: ", len(dataset.classes))
    elif data_split == 'test':
        test_transform = transforms.Compose([
            transforms.Resize(resize_res),
            transforms.CenterCrop(model_input_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_vec, std=std_vec)
        ])
        dataset = CassavaTestFolder(
            root=dir_path + competition_root_path + '/test', transform=test_transform)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=1, shuffle=False,
            num_workers=4, pin_memory=True)
        print("Number of test samples: ", len(dataset))

    return dataset, loader


In [0]:
#####resnet#####
import os
import torch
import torch.nn as nn
from torch.utils.model_zoo import load_url as load_state_dict_from_url


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.num_classes = num_classes
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)

        return x

    def load_pretrained_network(self, arch, dataset_numclasses):
        print('Loading pretrained model from {}..', model_urls[arch])
        state_dict = load_state_dict_from_url(model_urls[arch], progress=True)
        self.load_state_dict(state_dict)
        if self.num_classes != dataset_numclasses:
            self.num_classes = dataset_numclasses
            self.fc = nn.Linear(self.fc.in_features, dataset_numclasses)

    def load_finetuned_network(self, dataset_numclasses, state_dict):
        if self.num_classes != dataset_numclasses:
            self.num_classes = dataset_numclasses
            self.fc = nn.Linear(self.fc.in_features, dataset_numclasses)
        self.load_state_dict(state_dict)


model_params = {
    'block': {
        'resnet18': BasicBlock, 'resnet34': BasicBlock, 'resnet50': Bottleneck,
        'resnet101': Bottleneck, 'resnet152': Bottleneck,
    },
    'planes': {
        'resnet18': [2, 2, 2, 2], 'resnet34': [3, 4, 6, 3], 'resnet50': [3, 4, 6, 3],
        'resnet101': [3, 4, 23, 3], 'resnet152': [3, 8, 36, 3],
    },
}

def resnet(arch, dataset_numclasses, state_dict=None, pretrained=True, **kwargs):
    model = ResNet(model_params['block'][arch], model_params['planes'][arch], **kwargs)
    if state_dict:
        model.load_finetuned_network(dataset_numclasses, state_dict)
    elif pretrained:
        model.load_pretrained_network(arch, dataset_numclasses)
    return model


In [0]:
def generate_model(arch, dataset_numclasses, state_dict=None, use_cuda=True):
    if arch == 'inceptionv4':
        model = inceptionv4(dataset_numclasses, state_dict)
    elif arch == 'resnet34':
        model = resnet(arch, dataset_numclasses, state_dict)
    if use_cuda:
        model.cuda()
    return model


In [0]:
# Train the model
import os
import torch
import torch.nn as nn

use_cuda = True
model_input_size = 560
batch_size = 8
num_epochs = 100
arch = 'resnet34'
optim = 'sgd'
lr = 1e-2
train_percentage = 1
train = True
use_extraimages = False
validate = False
test = True
use_weighted_loss = False
tencrop_test = False
resume_path = ''
subset_finetune = False


def main():

    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.enabled = True
    # print("Model input size: ", model_input_size)
    # print("Batch size: ", batch_size)
    # print('Arch: ', arch)
    # print('Optimizer: ', optim)
    # print('Weighted loss: ', use_weighted_loss)

    dir_path = os.path.dirname('__file__')

    print('Loading dataset and dataloader..')
    train_percent = train_percentage
    if validate and train_percent == 1:
        train_percent = 0.9
        print('Warning: train percentage was given 1 with validation enabled, train percentage dropped to 0.9')
    train_set, train_loader = get_dataloader(data_split='train', train_percentage=train_percent)
    if validate:
        val_set, val_loader = get_dataloader(data_split='val', train_percentage=train_percent)
    if test:
        test_set, test_loader = get_dataloader(data_split='test')
    num_classes = len(train_set.classes)

    class_weights = [1.0] * num_classes
    if use_weighted_loss:
        class_weights = train_set.class_weights
        print("Class weights: ", class_weights)
    class_weights = torch.Tensor(class_weights)

    criterion = nn.CrossEntropyLoss(class_weights)
    if use_cuda:
        criterion = criterion.cuda()

    best_perf1 = 0
    best_perf5 = 0
    begin_epoch = 0
    best_epoch = 0
    state_dict = None  # won't be None if resuming from a trained model
    optimizer_dict = None  # won't be None if resuming from a trained model
    scheduler_steps = 2  # [30, 60, 90]
    scheduler_decay = 0.9  # 0.1

    if resume_path:
        print('Loading finetuned model from {}..',resume_path)
        checkpoint = torch.load(resume_path)
        begin_epoch = checkpoint['epoch']
        best_epoch = begin_epoch
        # best_epoch = checkpoint['best_epoch']
        best_perf1 = checkpoint['perf1']
        # best_perf5 = checkpoint['perf5']
        arch = checkpoint['arch']
        num_classes = checkpoint['num_classes']
        state_dict = checkpoint['state_dict']
        optimizer_dict = checkpoint['optimizer']
        print('Begin epoch: ', begin_epoch)
        print('Best Acc@1 at epoch {}: {}'.format(best_epoch, best_perf1))
        # scheduler.load_state_dict(checkpoint['scheduler'])
    # arch = 'inceptionv4'
    arch = 'resnet34'
    model = generate_model(arch, num_classes, state_dict, use_cuda=True)
    optimizer = get_optimizer( model, optimizer_dict)
    print('Learning rate: {:1.5f}', optimizer.param_groups[0]['lr'])

    if train:
        for epoch in range(begin_epoch, num_epochs):
            print('Epoch: {} / {}'.format(epoch+1, num_epochs))

            perf_indicator1, perf_indicator5 = train_epoch(
                epoch, train_loader, model, criterion, optimizer, use_cuda=True)

            if validate:
                perf_indicator1, perf_indicator5 = validate_epoch(
                    val_loader, model, criterion, use_cuda=True)

            if perf_indicator1 >= best_perf1:
                best_perf1 = perf_indicator1
                best_perf5 = perf_indicator5
                best_epoch = epoch

                checkpoint_file = '{}_{}_{}_{}{}{}'.format(
                    model_input_size, arch, optim,
                    batch_size, '_subset' if subset_finetune else '',
                    '_weightedloss' if use_weighted_loss else '')

                save_checkpoint({
                    'epoch': epoch + 1,
                    'best_epoch': best_epoch + 1,
                    'perf1': best_perf1,
                    'perf5': best_perf5,
                    'arch': arch,
                    'num_classes': model.num_classes,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    # 'scheduler': scheduler.state_dict(),
                }, dir_path, is_best=True, filename=checkpoint_file)
            arch = 'resnet34'
            if (epoch+1) % 5 == 0:  # save model every 5 epochs
                checkpoint_file = 'Epoch{}_{}_{}_{}_{}{}{}'.format(
                    epoch + 1, model_input_size, arch, optim,
                    batch_size, '_subset' if subset_finetune else '',
                    '_weightedloss' if use_weighted_loss else '')

                save_checkpoint({
                    'epoch': epoch + 1,
                    'best_epoch': best_epoch + 1,
                    'perf1': best_perf1,
                    'perf5': best_perf5,
                    'arch': arch,
                    'num_classes': model.num_classes,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    # 'scheduler': scheduler.state_dict(),
                }, dir_path, filename=checkpoint_file)

            print('Epoch {} perf acc@1: {}, perf acc@5: {}'.format(
                epoch+1, perf_indicator1, perf_indicator5))
            print('Best perf acc@1: {}, perf acc@5: {} at epoch {}'.format(
                best_perf1, best_perf5, best_epoch+1))
            # scheduler.step(perf_indicator1)
            if epoch+1 < 100:
                adjust_learning_rate(optimizer, epoch+1,  steps=scheduler_steps, dec_rate=scheduler_decay)

    if test:
        test_cassava(test_loader, model, train_set.classes) 


if __name__ == '__main__':
    main()



Loading dataset and dataloader..
Transform resize resolution:  640
Number of training samples:  5656
Number of classes:  5
Transform resize resolution:  640
Number of test samples:  3774
Loading pretrained model from {}.. https://download.pytorch.org/models/resnet34-333f7ec4.pth


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth



Learning rate: {:1.5f} 0.01
Epoch: 1 / 100
Epoch: [1][  1/707]	Time  6.133 ( 6.133)	Data  3.819 ( 3.819)	Acc@1  12.50 ( 12.50)	Acc@5 100.00 (100.00)	Loss 1.8764e+00 (1.8764e+00)
Epoch: [1][178/707]	Time  0.333 ( 0.871)	Data  0.206 ( 0.732)	Acc@1  37.50 ( 49.72)	Acc@5 100.00 (100.00)	Loss 2.0387e+00 (1.4524e+00)
Epoch: [1][355/707]	Time  0.339 ( 0.843)	Data  0.212 ( 0.710)	Acc@1  62.50 ( 54.44)	Acc@5 100.00 (100.00)	Loss 1.0291e+00 (1.3090e+00)
Epoch: [1][532/707]	Time  0.321 ( 0.840)	Data  0.197 ( 0.709)	Acc@1  62.50 ( 55.78)	Acc@5 100.00 (100.00)	Loss 8.6096e-01 (1.2406e+00)
Epoch: [1][707/707]	Time  0.333 ( 0.837)	Data  0.210 ( 0.707)	Acc@1  50.00 ( 57.46)	Acc@5 100.00 (100.00)	Loss 1.2383e+00 (1.1796e+00)
[[  25  275   22  136    8]
 [  50  895   61  422   15]
 [  10  169  146  441    7]
 [  17  333  137 2162    9]
 [   6  150   22  116   22]]
Epoch 1 perf acc@1: 57.46110534667969, perf acc@5: 100.0
Best perf acc@1: 57.46110534667969, perf acc@5: 100.0 at epoch 1
Epoch: 2 / 100
Epo

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:116: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Processed test data:  1
Processed test data:  421
Processed test data:  841
Processed test data:  1261
Processed test data:  1681
Processed test data:  2101
Processed test data:  2521
Processed test data:  2941
Processed test data:  3361
Processed test data:  3774
